### Make a Crawler to get the Pictures form NMNS with Python3
http://www.nmns.edu.tw/web/learn/feature/
- [目標相簿 - goal album 01](http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13)
- [目標相簿 - goal album 02](http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=14)
- [目標相簿 - goal album 03](http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=15)

In [ ]:
import selenium
from selenium import webdriver
import urllib.request
import ssl

相簿效期只有3個月，故須判斷是否仍在效期內。（上傳日期：2018-07-29） 
the album is alive for 3 months , set urlInvalidDate to check.  
album upload date: 2018-07-29

----
#### note
- [time module](https://docs.python.org/3/library/time.html)
[中文教學](http://www.runoob.com/python/python-date-time.html)
    - time.time() : seconds since 12:00am, January 1, 1970
    - time.localtime() : show local current time in struct_time
    - time.asctime() : show time in "Www Mmm dd hh:mm:ss yyyy" format  
- [datetime module](https://docs.python.org/3/library/datetime.html#module-datetime)
[中文教學01](https://www.cnblogs.com/feeland/p/4551054.html)
[中文教學02](https://blog.csdn.net/mathcompfrac/article/details/76229385)
[中文教學03](http://blog.51cto.com/leejia/1577662)
    - date.today() : show local current time in datetime
    - datetime.datetime.strptime() : format string to datetime
    - (date)replace() : replace date with given values

In [3]:
import datetime
from datetime import date

uploadDate = "2018-07-29"
aliveMonth = 3
localtime = date.today()

uploadDatetime = datetime.datetime.strptime(uploadDate,"%Y-%m-%d")
replaceMonth = uploadDatetime.month + aliveMonth
urlInvalidDate = uploadDatetime.replace(month = replaceMonth)

依據網頁結構，發現2種圖片網址可能所在之 xpath & attribute 對應：  
two tpyes of the HTML structure.  

type 01
<code>
    < div   
        class="swiper-slide"  
        style="  
            background-image: url("
                https://wwwadmin.nmns.edu.tw/upload/activity/1532917261goKuy5PTED.JPG");  
            width: 767px;"  
        data-swiper-slide-index="58">  
    < /div>
</code>

type 02
<code>
    < div 
         class="swiper-slide" 
         style="text-align: center; width: 669px;" 
         data-swiper-slide-index="9">
         < img
             src="https://wwwadmin.nmns.edu.tw/upload/activity/1532917211bBTZsVMFkB.JPG" 
             height="100%" 
             style="max-width:100%;">
    < /div>
</code>

In [ ]:
xpath_attribute = {
    '//div[@data-swiper-slide-index]':'style',
    '//div[@data-swiper-slide-index]/img':'src'
}

# urls to be get
url_dict = {}

In [ ]:
driver = webdriver.Chrome(r'D:\pyLadies\2018_0805_nmnsCrawler\chromedriver.exe')
totalDays = 3

In [ ]:
# method : get the clean URLs, and put in url_dict{}
def findURL(xpath,attribute):
    images = driver.find_elements_by_xpath(xpath)
    index = 0 #圖片張數計算

    for image in images:
        tagContent = image.get_attribute(attribute)
        
        if tagContent.find('\"') != -1: tagContent = tagContent.split('\"')[-1]
            # ↑split the string if it comes from attribute "style"
        else: pass

        if tagContent.find("https://") != -1:
            index += 1
            url = tagContent.split('\"')[0]
            # ↑split to get clear url from attribute "style"
            if url not in url_dict:
                url_dict[url] = "Day" + str(day) + "_" + str(index).zfill(3) + ".jpg"
            else:
                pass
        else:
            return False

In [ ]:
# main
if localtime > urlInvalidDate: # check the date, future/now is larger than past
    print("相簿已逾期！請手動確認檔案是否仍存在。")
else:
    for day in range(1,totalDays+1): # get the url of the day
        urlDay = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id={idNo}'.format(idNo=12+day)
             # ↑url of day 1 = http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13
        driver.get(urlDay)

        for xpath,attribute in  xpath_attribute.items(): # get url from xpath
            if findURL(xpath,attribute) != False:
                print("Day",day,"取得網址！","目前字典長度:",len(url_dict))
            else:
                print( "注意：「", xpath, " + ", attribute, "」的組合失敗。" )
                  
    for key,value in url_dict.items(): #download images from url,and save in file name
#        print(key,value)
        ssl._create_default_https_context = ssl._create_unverified_context
            # ↑ to pass the SSL certificate verify
        resource = urllib.request.urlopen(key)
        output = open(value,"wb") # write binary
        output.write(resource.read())
        output.close()
    print("下載結束")

注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。  
Day 1 取得網址！ 目前字典長度: 85  
注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。  
Day 2 取得網址！ 目前字典長度: 363  
注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。  
Day 3 取得網址！ 目前字典長度: 475  
下載結束

In [ ]:
driver.close()